In [3]:
import joblib, json, pandas as pd, numpy as np
from pathlib import Path

# ---------- (1) input model and features ----------
model       = joblib.load('rf_model_0621.pkl')
features_r  = json.loads(Path('features_r.json').read_text())

# ---------- (2)  lookup  ----------
ZONE   = {z['zone_id']: z
          for z in json.loads(Path('zone_defaults.json').read_text())}
GLOBAL = json.loads(Path('global_defaults.json').read_text())

# ---------- (3) real-time data example ----------
request_json = {
    "zone_id": 33,
    "hour": 18,
    "weekday": 4,
    "month": 6,
    "day": 27,
    "is_weekend": 0,
    "temp": 29.4,
    "prcp": 0.0,
    "interest": 0.72
    
}

# ---------- (4) filled in fixed data ----------
def build_payload(req):
    z = req['zone_id']
    payload = req.copy()
    payload.update(ZONE.get(z, {}))
    for k, v in GLOBAL.items():
        payload.setdefault(k, v)
    return payload

payload = build_payload(request_json)

# ---------- (5) one-hot & align ----------
def onehot_align(df_raw, feature_list):
    df = pd.get_dummies(df_raw, prefix_sep='_')

    for col in feature_list:
        if col not in df.columns:
            df[col] = 0
    return df[feature_list]  

X = onehot_align(pd.DataFrame([payload]), features_r)

# ---------- (6) predict  ----------
y_pred_log = model.predict(X)[0]
y_pred     = np.expm1(y_pred_log)

print(f" total_flow prediction = {y_pred:,.1f} people")


 total_flow prediction = 3,777.4 people
